In [1]:
import os
import time

import numpy as np
import torch
import pandas as pd
import random
from torch.distributions.binomial import Binomial
from torch.distributions.bernoulli import Bernoulli
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader 

from transformers import DataCollatorForLanguageModeling, BertForMaskedLM
from transformers import Trainer, TrainingArguments

from tokens import WordLevelBertTokenizer
from vocab import create_vocab
from data import CausalBertDataset, MLMDataset
from causal_bert import CausalBOW

In [2]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = '3'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
def true_casual_effect(data_loader, effect='ate', estimation='Q'):
    dataset = data_loader.dataset
    
    Q1 = dataset.treatment * dataset.response + (1 - dataset.treatment) * dataset.pseudo_response
    Q1 = Q1.cpu().data.numpy().squeeze()

    Q0 = dataset.treatment * dataset.pseudo_response + (1 - dataset.treatment) * dataset.response
    Q0 = Q0.cpu().data.numpy().squeeze()

    treatment = dataset.treatment.cpu().data.numpy().squeeze()
    prop_scores = dataset.prop_score.cpu().data.numpy().squeeze()
    
    if estimation == 'q':
        if effect == 'att':
            phi = (treatment * (Q1 - Q0))
            return phi.sum() / treatment.sum()
        elif effect == 'ate':
            return (Q1 - Q0).mean()
        
    elif estimation == 'plugin':
        phi = (prop_scores * (Q1 - Q0)).mean()
        if effect == 'att':
            return phi / treatment.mean()
        elif effect == 'ate': 
            return phi

def est_casual_effect(data_loader, model, effect='ate', estimation='Q'):
    # We use `real_treatment` here to emphasize the estimations use real instead of estimated treatment.
    real_response, real_treatment = [], []
    prop_scores, Q1, Q0 = [], [], []

    model.eval()
    for idx, (tokens, treatment, response) in enumerate(data_loader):
        real_response.append(response.cpu().data.numpy().squeeze())
        real_treatment.append(treatment.cpu().data.numpy().squeeze())

        prop_score, q1, q0 = model(tokens)

        prop_scores.append(prop_score.cpu().data.numpy().squeeze())
        Q1.append(q1.cpu().data.numpy().squeeze())
        Q0.append(q0.cpu().data.numpy().squeeze())

    real_response = np.concatenate(real_response, axis=0)
    real_treatment = np.concatenate(real_treatment, axis=0)

    Q1 = np.concatenate(Q1, axis=0)
    Q0 = np.concatenate(Q0, axis=0)
    prop_scores = np.concatenate(prop_scores, axis=0)
    
    model.train()
    
    if estimation == 'q':
        if effect == 'att':
            phi = (real_treatment * (Q1 - Q0))
            return phi.sum() / real_treatment.sum()            
        elif effect == 'ate':
            return (Q1 - Q0).mean()
        
    elif estimation == 'plugin':
        phi = (prop_scores * (Q1 - Q0)).mean()
        if effect == 'att':
            return phi / real_treatment.mean()
        elif effect == 'ate':
            return phi

# Prepare data

In [4]:
vocab = create_vocab(merged=True, uni_diag=True)

tokenizer = WordLevelBertTokenizer(vocab)

alpha = 0.75
beta = 25.
c = 0.2
i = 0.

In [5]:
start = time.time()
trainset = CausalBertDataset(tokenizer=tokenizer, data_type='merged', is_unidiag=True,
                             alpha=alpha, beta=beta, c=c, i=i, 
                             group=list(range(1)), max_length=512, min_length=10,
                             truncate_method='first', device=device)

print(f'Load training set in {(time.time() - start):.2f} sec')

Load training set in 61.56 sec


In [6]:
start = time.time()
testset = CausalBertDataset(tokenizer=tokenizer, data_type='merged', is_unidiag=True,
                            alpha=alpha, beta=beta, c=c, i=i, 
                            group=[9], max_length=512, min_length=10,
                            truncate_method='first', device=device)

print(f'Load training set in {(time.time() - start):.2f} sec')

Load training set in 60.25 sec


In [7]:
train_loader = DataLoader(trainset, batch_size=64, drop_last=True, shuffle=True)
test_loader = DataLoader(testset, batch_size=1024, drop_last=True, shuffle=True)

# Create and train a Causal-Bert

In [8]:
# trained_bert = '/nfs/turbo/lsa-regier/bert-results/results/behrt/MLM/merged/unidiag/checkpoint-4574003/'
trained_bert = '/home/liutianc/emr/bert/results/behrt/MLM/merged/unidiag/checkpoint-6018425/'

model = BertForMaskedLM.from_pretrained(trained_bert)
token_embed = model.get_input_embeddings()

model = CausalBOW(token_embed, learnable_docu_embed=False, hidden_size=64).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

q_loss = nn.BCELoss()
prop_score_loss = nn.BCELoss()

# Please specify the effect and estimation we want to use here.
effect = 'ate'
estimation = 'q'

effect = effect.lower()
estimation = estimation.lower()
assert effect in ['att', 'ate'], f'Wrong effect: {effect}...'
assert estimation in ['q', 'plugin'], f'Wrong estimation: {estimation}...'

In [9]:
real_att_q = true_casual_effect(test_loader, effect, estimation)

print(f'Real: [effect: {effect}], [estimation: {estimation}], [value: {real_att_q:.5f}]')
print(f'unadjusted: {(testset.response[testset.treatment == 1].mean() - testset.response[testset.treatment == 0].mean()).item():.4f}')

Real: [effect: ate], [estimation: q], [value: 0.16753]
unadjusted: 0.2394


In [10]:
epoch = 10
rs_loss, rq1_loss, rq0_loss = [0.] * 3

for e in range(1, epoch + 1):
    model.train()
    start = time.time()
    for idx, (tokens, treatment, response) in enumerate(train_loader):
        optimizer.zero_grad()
        prop_score, q1, q0 = model(tokens)
        
        s_loss  = prop_score_loss(prop_score, treatment)
        q1_loss = q_loss(q1[treatment==1], response[treatment==1])
        q0_loss = q_loss(q0[treatment==0], response[treatment==0])
        
        s_loss.backward(retain_graph=True)
        q1_loss.backward(retain_graph=True)
        q0_loss.backward()
        optimizer.step()
        
        rs_loss  += s_loss.item()
        rq1_loss += q1_loss.item()
        rq0_loss += q0_loss.item()
        
    run_idx = idx

    # Evaluation.
    train_est_att_q = est_casual_effect(train_loader, model, effect, estimation)
    test_est_att_q = est_casual_effect(test_loader, model, effect, estimation)
    
    print(f'''epoch: {e} / {epoch}, time cost: {(time.time() - start):.2f} sec, 
          g_loss: {(rs_loss / (run_idx + 1))  :.5f}, q1_loss: {(rq1_loss / (run_idx + 1)):.5f}, q0_loss: {(rq0_loss/ (run_idx + 1)) :.5f}, 
          Effect: [effect: {effect}], [estimation: {estimation}], [train: {train_est_att_q:.5f}], [test: {test_est_att_q:.5f}]''')
    start = time.time()
    rs_loss, rq1_loss, rq0_loss = [0.] * 3

print('Finish training...')

# With only 3 groups to train.

epoch: 1 / 10, time cost: 71.51 sec, 
          g_loss: 0.87506, q1_loss: 0.56730, q0_loss: 0.73112, 
          Effect: [effect: ate], [estimation: q], [train: 0.20333], [test: 0.20352]
epoch: 2 / 10, time cost: 74.69 sec, 
          g_loss: 0.54121, q1_loss: 0.56026, q0_loss: 0.69944, 
          Effect: [effect: ate], [estimation: q], [train: 0.25347], [test: 0.25370]
epoch: 3 / 10, time cost: 74.58 sec, 
          g_loss: 0.52849, q1_loss: 0.55487, q0_loss: 0.69674, 
          Effect: [effect: ate], [estimation: q], [train: 0.19163], [test: 0.19196]
epoch: 4 / 10, time cost: 70.21 sec, 
          g_loss: 0.52099, q1_loss: 0.55004, q0_loss: 0.69503, 
          Effect: [effect: ate], [estimation: q], [train: 0.22371], [test: 0.22409]
epoch: 5 / 10, time cost: 79.20 sec, 
          g_loss: 0.51626, q1_loss: 0.54709, q0_loss: 0.69349, 
          Effect: [effect: ate], [estimation: q], [train: 0.23198], [test: 0.23232]
epoch: 6 / 10, time cost: 74.60 sec, 
          g_loss: 0.51306, q1_lo

In [11]:
effect = 'att'
real_att_q = true_casual_effect(test_loader, effect, estimation)
est_att_q = est_casual_effect(test_loader, model, effect, estimation)

print(f'Real: [effect: {effect}], [estimation: {estimation}], [value: {real_att_q:.5f}]')
print(f'unadjusted: {(testset.response[testset.treatment == 1].mean() - testset.response[testset.treatment == 0].mean()).item():.4f}')
print(f'Estimated: [effect: {effect}], [estimation: {estimation}], [value: {est_att_q:.5f}]')

Real: [effect: att], [estimation: q], [value: 0.13733]
unadjusted: 0.2394
Estimated: [effect: att], [estimation: q], [value: 0.18985]


In [12]:
effect = 'ate'
real_att_q = true_casual_effect(test_loader, effect, estimation)
est_att_q = est_casual_effect(test_loader, model, effect, estimation)

print(f'Real: [effect: {effect}], [estimation: {estimation}], [value: {real_att_q:.5f}]')
print(f'unadjusted: {(testset.response[testset.treatment == 1].mean() - testset.response[testset.treatment == 0].mean()).item():.4f}')
print(f'Estimated: [effect: {effect}], [estimation: {estimation}], [value: {est_att_q:.5f}]')

Real: [effect: ate], [estimation: q], [value: 0.16753]
unadjusted: 0.2394
Estimated: [effect: ate], [estimation: q], [value: 0.18025]
